# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. 

**Цель исследования** - разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.

Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Ход исследования**

Данные о статистике платёжеспособности клиентов получим из файла `data.csv`. О качестве данных ничего не известно. Поэтому перед проверкой гипотезы и ответом на вопросы понадобится обзор данных.

Проверим данные на ошибки и оценим их влияние на исследование. Далее, на этапе предобработки, поищем возможности исправить критические ошибки данных.

Таким образом, исследование пройдет в 4 этапа:
1. [Обзор данных.](#intro) 
2. [Предобработка данных.](#intro1) 
3. [Ответы на вопросы.](#intro2)
4. [Выводы.](#intro3)

# 1. Обзор данных  <a id='intro'></a>

In [1]:
# импорт бибилиотеки pandas
import pandas as pd
# импорт библиотеки pymystem3
from pymystem3 import Mystem
# импорт библиотеки collections
from collections import Counter

In [2]:
# чтение файла с данными и сохранение в df
try:
    df = pd.read_csv('/datasets/data.csv')
except:
    df = pd.read_csv('data.csv')    

In [3]:
# получение первых 15 строк таблицы df
df.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
# получение общей информации о данных в таблице df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [5]:
# получение информации об описательных статистических данных 
# в таблице df
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Итак, в таблице 12 столбцов. У столбцов наблюдается 3 типа данных: `float64`, `int64`, `object`.

Согласно документации к данным: 
* `children` — количество детей в семье;
* `days_employed` — общий трудовой стаж в днях;
* `dob_years` — возраст клиента в годах;
* `education` — уровень образования клиента;
* `education_id` — идентификатор уровня образования;
* `family_status` — семейное положение;
* `family_status_id` — идентификатор семейного положения;
* `gender` — пол клиента;
* `income_type` — тип занятости;
* `debt` — имел ли задолженность по возврату кредитов;
* `total_income` — ежемесячный доход;
* `purpose` — цель получения кредита.

В названиях столбцов отсутсвуют нарушения стиля. 

Однако, в столбцах `days_employed` и `total_income` присутствует меньшее количество значений, чем в остальных столбцах, следовательно, в данных есть пропущенные значения. В столбце `days_employed` необходимо поработать над чистотой данных, а именно перевести их в положительные значения и перевести аномальные значения в дни, так как одни значения представлены в часах, а другие в днях. В столбце `total_income` необходимо значения уменьшить, то есть преобразовать цифры до двух знаков после запятой.

Также в столбце `edication` нет единого регистра, необходимо привести его к нижнему регистру, чтобы данные выглядели одинаково, в едином формате.

**Вывод**

В каждом столбце представлена информация о конкретном человеке для получения кредита, а именно: сколько детей в семье, трудовой стаж, возраст, образование, семейное положение, пол, тип занятости, доход. 
А также, информация в двух столбцах, которая относится непосредственно к получению кредита и кредитной истории человека: имелась ли задолженность по возврату кредитов и цель получения кредита.

Предварительно, можно утверждать, что данных достаточно для проверки гипотезы и ответов на вопросы. Но, встречаются пропуски в данных, нет единого регистра с столбце `education`, также в столбце `days_employed` необходимо поработать над чистотой входных значений и в столбце `total_income` уменьшить в доходах количество знаков после запятой.

Чтобы приступить к ответам на вопросы необходимо устранить проблемы в данных.

# 2. Предобработка данных <a id='intro1'></a>

### Обработка пропусков

In [6]:
# подсчёт пропущенных значений

df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Возможно, что пропуски в столбце `days_employed` возникли по причине того, что выгрузка была совершена из двух разных систем, так как значения по стажу представлены и в часах, и в днях.

В столбцах `days_employed` и `total_income` присутствуют данные, которые необходимы для дальнейшего анализа. Поэтому, для сохранения всех данных и более точного анализа нужно пропущенные значения заменить на такие же количественные значения с помощью медианы.

Но, чтобы приступить к замене пропущенных значений, нужно в столбце `days_employed` привести данные к единому формату, а именно сделать числа положительными, а также перевести все значения в дни. 

In [7]:
# перевод значений столбца 'days_employed' в положительные значения 

df['days_employed'] = df['days_employed'].abs() 

In [8]:
# перевод значений столбца 'days_employed' в дни

df.loc[df['days_employed'] > 30000, 'days_employed'] = df.loc[df['days_employed'] > 30000, 'days_employed'] / 24

In [9]:
# Код ревьюера
(df[df['income_type'] == 'пенсионер']['dob_years'] - (df[df['income_type'] == 'пенсионер']['days_employed'] / 365)).describe()

count    3443.000000
mean       17.470025
std         7.889311
min       -45.775385
25%        13.807519
50%        17.979299
75%        22.234049
max        35.012771
dtype: float64

In [10]:
# замена пропущенных значений в столбце 'days_employed' на медианное значение

print('Пропуски до:', df['days_employed'].isna().sum())
df.loc[df['days_employed'].isna(), 'days_employed'] = df['days_employed'].median()
print('Пропуски после:', df['days_employed'].isna().sum())

Пропуски до: 2174
Пропуски после: 0


In [11]:
(df[df['income_type'] == 'пенсонер']['dob_years'] - (df[df['income_type'] == 'пенсонер']['days_employed'] / 365)).mean()

nan

In [12]:
# замена пропущенных значений столбца 'total_income' на медианное значение

print('Пропуски до:', df['total_income'].isna().sum())
df.loc[df['total_income'].isna(), 'total_income'] = df['total_income'].median()
print('Пропуски после:', df['total_income'].isna().sum())

Пропуски до: 2174
Пропуски после: 0


In [13]:
# преобразование значений в столбце 'total_income' до двух знаков после запятой
# и просмотр результата

df['total_income'] = df['total_income'].round(2)
df['total_income']    

0        253875.64
1        112080.01
2        145885.95
3        267628.55
4        158616.08
           ...    
21520    224791.86
21521    155999.81
21522     89672.56
21523    244093.05
21524     82047.42
Name: total_income, Length: 21525, dtype: float64

**Вывод**

 

Таким образом, для сохранения данных и более точного анализа, мы:

- заменили пропуски в столбцах `days_employed` и `total_income`на медианные значения;

- в столбце `days_employed` перевели значения из отрицательных в положительные и привели все данные к единому формату (все значения стали показываться в формате дней);

- в столбце `total_income` уменьшили значения дохода до двух знаков после запятой.

### Замена типа данных

In [14]:
# перевод значений столбца 'days_employed' в тип данных int

df['days_employed'] = df['days_employed'].astype('int')

In [15]:
# просмотр первых 5 строк столбца 'days_employed'

df['days_employed'].head()

0     8437
1     4024
2     5623
3     4124
4    14177
Name: days_employed, dtype: int64

**Вывод**

В столбце `days_employed` мы заменили вещественный тип данных на целочислительный с помощью метода `astype()`. Данный метод позволяет перевести значения именно в тип данных `int`.

Таким образом, значения в столбце `days_employed` пригодны для дальнейших ответов на вопросы по анализу данных.

Далее, в следующем пункте, необходимо установить в таблице наличие дубликатов, как явных, так и неявных. 

### Обработка дубликатов

Однако, перед тем, как начнем подсчитывать дубликаты в таблице, необходимо привести значения в столбце `education` к нижнему регистру, так как при дальнейшем нахождении дубликатов значения в разном регистре засчитаются как разные.

In [16]:
# подсчет уникальных значений столбца 'education'

df['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [17]:
# приведение строки к нижнему регистру
# и подсчет уникальных значений столбца 'education'

df['education'] = df['education'].str.lower()
df['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Посчитаем количество явных дубликатов в таблице.

In [18]:
print('Количество явных дубликатов до:', df.duplicated().sum())
df = df.drop_duplicates().reset_index(drop = True)
print('Количество явных дубликатов после:', df.duplicated().sum())

Количество явных дубликатов до: 71
Количество явных дубликатов после: 0


In [19]:
# подсчет уникальных значений столбца 'children'

df['children'].value_counts()

 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

В столбце `children` мы обнаружили такое количество детей, как `-1` и `20`. Возможно, что при выгрузке данных произошла ошибка. Заменим число `-1` на `0`, а число `20` на медианное значение, так как мы не знаем случайно ли ввели лишний ноль или вместо 0 случайно поставили 20.

In [20]:
# замена значения '-1' на '0' в столбце 'children'

df['children'] = df['children'].replace(-1, 0)

In [21]:
# замена числа 20 на медианное значение в столбце 'children'
# и просмотр уникальных значений столбца

children_median = df[df['children'] != 20]['children'].median()
df['children'] = df['children'].replace(20, children_median)
df['children'].value_counts() 

0    14214
1     4808
2     2052
3      330
4       41
5        9
Name: children, dtype: int64

In [22]:
# просмотр уникальных значений столбца 'dop_years'

df['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

После просмотра уникальных значений в столбце `dob_years` мы обнаружили, что среди всех возрастов присутствует число `0`. Возможно, что произошла также ошибка при выгрузке данных из системы. Поэтому, самым оптимальным решением, будет опять же замена числа `0` на медианное значение.

In [23]:
# замена в столбце 'dob_years' числа '0' на медианное значение
# и просмотр уникальных значений с помощью метода unique()

dop_years_median = df[df['dob_years'] != 0]['dob_years'].median()
df['dob_years'] = df['dob_years'].replace(0, dop_years_median)
df['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51, 59, 29, 60, 55, 58, 71, 22, 73, 66,
       69, 19, 72, 70, 74, 75])

In [24]:
# просмотр уникальных значений столбца 'family_status'

df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

При просмотре уникальных значений столбца `family_status` мы обнаружили, что в значении `Не женат / не замужем` первая буква `Н` прописная. Необходимо привести все значения к строчным буквам.

In [25]:
# приведение значений в столбце 'family_status' к нижнему регистру
# и просмотр уникальных значений данного столбца

df['family_status'] = df['family_status'].str.lower()
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

In [26]:
# просмотр уникальных значений столбца 'gender'

df['gender'].value_counts()

F      14174
M       7279
XNA        1
Name: gender, dtype: int64

При просмотре уникальных значений в столбце `gender` обнаружили значение `XNA`. Скорее всего, это пропуск, но так как мы не можем определить пол человека и по причине того, что данный столбец не так важен нам в дальнейшем анализе данных и ответах на вопросы, мы не будем заменять данное значение и оставим таким. 

Также, данное значение встречается только один раз, поэтому оно не сильно скажется на итоговых результатах анализа.

**Вывод**

В процессе поиска и удаления дубликатов были рассмотрены, как явные и так и неявные дубликаты. Сначала мы использовали методы для поиска и удаления явных дубликатов во всей таблице.
Далее просматривали каждый столбец отдельно, чтобы определить неявные дубликаты:
* в столбце `education` мы привели все строки к нижнему регистру;
* в столбце `children` заменили число `-1` на `0` и `20` на медианное значение;
* в столбце `dob_years`, где указан возраст клиентов, заменили `0`на медианное значение;
* в столбце `family_status` привели значение `Не женат / не замужем` к нижнему регистру.

Также, при поиске уникальных значений в столбце `gender` было обнаружено значение `XNA`. С ним мы ничего не стали делать, так как оно встречается всего лишь один раз и никак не влияет на итоговые результаты исследования.

Возможно, что неявные дубликаты и пропуск в форме `XNA` возникли по причине ошибок в процессе выгрузки данных.

В следующем пункте необходимо произвести классификацию столбца `purpose`. Для этого нам понадобится лемматизация.

### Лемматизация

In [28]:
# просмотр уникальных значений столбца 'purpose'

df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [29]:
# объединение значений в столбце 'purpose' в строку
#  и лемматизация данной строки

m = Mystem()
purpose_list = ','.join(df['purpose'].unique())
lemmas = m.lemmatize(purpose_list)
print(lemmas) 

['покупка', ' ', 'жилье', ',', 'приобретение', ' ', 'автомобиль', ',', 'дополнительный', ' ', 'образование', ',', 'сыграть', ' ', 'свадьба', ',', 'операция', ' ', 'с', ' ', 'жилье', ',', 'образование', ',', 'на', ' ', 'проведение', ' ', 'свадьба', ',', 'покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', ',', 'покупка', ' ', 'недвижимость', ',', 'покупка', ' ', 'коммерческий', ' ', 'недвижимость', ',', 'покупка', ' ', 'жилой', ' ', 'недвижимость', ',', 'строительство', ' ', 'собственный', ' ', 'недвижимость', ',', 'недвижимость', ',', 'строительство', ' ', 'недвижимость', ',', 'на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', ',', 'на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', ',', 'операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', ',', 'строительство', ' ', 'жилой', ' ', 'недвижимость', ',', 'жилье', ',', 'операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', ',', 'автомобиль', ',', 'заниматься', ' ', 'образование', ',', 'сделка', ' ', 'с', ' ', 'подержать

In [30]:
# подсчет частоты появления слов с помошью Counter

print(Counter(lemmas))

Counter({' ': 59, ',': 37, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'подержать': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'со': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1, '\n': 1})


Таким образом можно выделить ключевые слова, которые обобщают все представленные цели получения кредита. 
Это будут следующие значения:
* недвижимость/жилье;
* автомобиль;
* образование;
* свадьба;
* ремонт.

In [31]:
# создание функции 'find_purpose'

def find_purpose(purpose):
    '''
    Функция последовательно лемматизирует каждое значение столбца 'purpose'
    и проверяет, какая из категорий обнаруживается в лемматизированном варианте
    '''
    purpose_lemmas = m.lemmatize(purpose)
    if 'недвижимость' in purpose_lemmas or 'жилье' in purpose_lemmas:
        return 'недвижимость'
    if 'автомобиль' in purpose_lemmas:
        return 'автомобиль'
    if 'образование' in purpose_lemmas:
        return 'образование'
    if 'свадьба' in purpose_lemmas:
        return 'свадьба'
    if 'ремонт' in purpose_lemmas:
        return 'ремонт'
    return 'прочее'

# созданиe столбца 'purpose_categories' и добавление значений с помощью метода 'apply'

df['purpose_categories'] = df['purpose'].apply(find_purpose)
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_categories
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.64,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.01,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.95,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.55,дополнительное образование,образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.08,сыграть свадьбу,свадьба


**Вывод**

Мы провели лемматизацию и выявили ключевые слова в столбце `purpose`. Всего было выявлено `5` значений: `недвижимость/жилье`; `автомобиль`; `образование`; `свадьба`; `ремонт`. Далее мы создали новый столбец `purpose_categories` в котором представлены основные категории целей получения кредита.

В следующем пункте необходимо произвести категоризацию по возрастным группам столбца `dob_years` и категоризацию по доходам столбца `total_income`.

### Категоризация данных

In [32]:
# просмотр уникальных значений столбца 'dob_years'

df['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51, 59, 29, 60, 55, 58, 71, 22, 73, 66,
       69, 19, 72, 70, 74, 75])

In [33]:
# создание функции для классификации людей по возрастным группам

def years_group(age):
    '''
    Возвращает возрастную группу по значению возраста age, используя правила:
    - 'молодые', если age более 18 и менее 44, включая 18 и 44;
    - 'средние', если age более 45 и менее 59, включая 45 и 59;
    - 'пожилые' - во всех остальных случаях.
    '''
    if age >= 18 and age <= 44:
        return 'молодые'
    if age >= 45 and age <= 59:
        return 'средние'
    return 'пожилые'

# создание столбца 'years_group'
# и просмотр первых пяти строк таблицы df

df['years_group'] = df['dob_years'].apply(years_group)
df.head(5)           

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_categories,years_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.64,покупка жилья,недвижимость,молодые
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.01,приобретение автомобиля,автомобиль,молодые
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.95,покупка жилья,недвижимость,молодые
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.55,дополнительное образование,образование,молодые
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.08,сыграть свадьбу,свадьба,средние


In [34]:
# просмотр уникальных значений столбца 'years_group' и их количества

df['years_group'].value_counts()

молодые    11808
средние     7146
пожилые     2500
Name: years_group, dtype: int64

In [35]:
# просмотр уникальных значений столбца 'total_income'

df['total_income'].sort_values()

14555      20667.26
12983      21205.28
16138      21367.65
1598       21695.10
14247      21895.61
            ...    
17137    1711309.27
20742    1715018.39
9159     1726276.01
19548    2200852.21
12390    2265604.03
Name: total_income, Length: 21454, dtype: float64

По уникальным значениям и их количеству мы видим, что минимальное значение по доходу составляет `20667.26`, а максимальное `2265604.03`. Следовательно можно определить границы для категоризации по доходу, отталкиваясь от минимального и максимального значения в таблице и далее разбить доход по статистическим данным зарплатной сетки в обществе.

In [36]:
# создание функции для классификации доходов людей

def total_income_group(number):
    '''
    Возвращает группу по значению дохода number, используя правила:
    - '1', если number менее 55000;
    - '2', если number более 55000 и менее 120000, включая 55000 и 120000;
    - '3', если number более 130000 и менее 200000, включая 130000 и 200000;
    - '4' - во всех остальных случаях.
    '''
    if number < 55000:
        return 1
    if number >= 55000 and number <= 120000:
        return 2
    if number >= 130000 and number <= 200000:
        return 3
    return 4

# создание столбца 'total_income_group'
# и просмотр первых пяти строк таблицы df

df['total_income_group'] = df['total_income'].apply(total_income_group)
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_categories,years_group,total_income_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.64,покупка жилья,недвижимость,молодые,4
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.01,приобретение автомобиля,автомобиль,молодые,2
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.95,покупка жилья,недвижимость,молодые,3
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.55,дополнительное образование,образование,молодые,4
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.08,сыграть свадьбу,свадьба,средние,3


In [37]:
# Код ревьюера

df.groupby('total_income_group').agg(count=('total_income','count'),std=('total_income','std'))

,count,std
total_income_group,,
1,580,7532.015890
2,6265,17390.860512
3,8446,18558.507784
4,6163,128015.773258


**Вывод**

Таким образом, мы провели категоризацию по возрастным группам и создали столбец `years_group`. В процессе категоризации были выявлены следующие основные возрастные группы:
* молодые (возраст от 18 до 44 включительно);
* средние (возраст от 45 до 59 включительно);
* пожилые (все остальные возраста)

Также, мы провели категоризацию по доходам людей и создали столбец `total_income_group`. В процессе категоризации были выявлены следующие основные группы по доходам людей:
* 1 (доход до 55000, не включая);
* 2 (доход от 55000 до 120000 включительно);
* 3 (доход от 130000 до 200000 включительно);
* 4 (все остальные доходы)

Мы определили и заполнили пропущенные значения, изменили тип данных в столбце `days_employed`, удалили дубликаты, выделили леммы в значениях столбца `purpose`, категоризировали данные. 

Теперь можно приступить к ответам на вопросы.

# 3. Ответы на вопросы <a id='intro2'></a>

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [38]:
# создание функции для категоризации клиентов по детям

def children_exist(children):
    '''
    Возвращает группу по количеству детей children, используя правила:
    - '0', если children равно 0;
    - '1', во всех остальных случаях.
    '''
    if children == 0:
        return 0
    return 1

# создание столбца 'children_exist'
df['children_exist'] = df['children'].apply(children_exist)

In [39]:
# создание сводной таблицы

data_pivot_children_exist_debt = df.pivot_table(index = 'children_exist', columns = 'debt', 
                                                values = 'gender', aggfunc = 'count')
data_pivot_children_exist_debt

debt,0,1
children_exist,,
0,13142,1072
1,6571,669


In [40]:
# подсчет вероятности задолженности для каждого человека с наличием или отсутствием детей
# и создание столбца 'ratio'

data_pivot_children_exist_debt['ratio'] = round(data_pivot_children_exist_debt[1] / (
    data_pivot_children_exist_debt[0] + data_pivot_children_exist_debt[1]), 3)
data_pivot_children_exist_debt.sort_values(by = 'ratio', ascending = False)

debt,0,1,ratio
children_exist,,,
1,6571,669,0.092
0,13142,1072,0.075


In [41]:
# Код ревьюера

df.groupby(df['children'] != 0).agg(число_клиентов=('debt','count'), число_должников=('debt','sum'), доля=('debt','mean')).\
                                    style.format({'доля' : '{:.2%}'.format})

,число_клиентов,число_должников,доля
children,,,
False,14214,1072,7.54%
True,7240,669,9.24%


**Вывод**

Согласно сводной таблице, наличие детей увеличивает вероятность задолженности по кредиту, чем наоборот их отсутствие.
Можно предположить, что, если возникает какая то сложная финансовая ситуация, людям без детей легче распределить свой бюджет и вовремя погасить кредит, чем людям с детьми, потому что им нужно обеспечить одного своего ребенка или несколько всем необходимым.

Следовательно, можно сказать, что зависимости между наличием детей и возвратом кредита в срок не существует.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [42]:
# # создадим сводную таблицу

data_pivot_family_status_debt = df.pivot_table(index = 'family_status', columns = 'debt', 
                                               values = 'years_group', aggfunc = 'count')
data_pivot_family_status_debt

debt,0,1
family_status,,
в разводе,1110,85
вдовец / вдова,896,63
гражданский брак,3763,388
женат / замужем,11408,931
не женат / не замужем,2536,274


In [43]:
# подсчет вероятности задолженности для каждого вида семейного положения
# и создание столбца 'ratio'

data_pivot_family_status_debt['ratio'] = round(data_pivot_family_status_debt[1] / (
    data_pivot_family_status_debt[0] + data_pivot_family_status_debt[1]), 3)
data_pivot_family_status_debt.sort_values(by = 'ratio', ascending = False)

debt,0,1,ratio
family_status,,,
не женат / не замужем,2536,274,0.098
гражданский брак,3763,388,0.093
женат / замужем,11408,931,0.075
в разводе,1110,85,0.071
вдовец / вдова,896,63,0.066


**Вывод**

Согласно сводной таблице, если люди ни разу не состояли в официальном браке, то вероятность задолженности по кредиту будет выше, чем у людей, которые хоть один раз были в браке. Следует также отметить, что люди у которых супруг/супруга скончались, чаще всех платят в срок по кредиту.

Следовательно, можно сказать, что зависимость между семейным положением и возвратом кредита в срок существует.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [44]:
# создание сводной таблицы

data_pivot_total_income_group_debt = df.pivot_table(index = 'total_income_group', columns = 'debt',
                                                    values = 'gender', aggfunc = 'count')
data_pivot_total_income_group_debt 

debt,0,1
total_income_group,,
1,545,35
2,5749,516
3,7709,737
4,5710,453


In [45]:
# подсчет вероятности задолженности для каждого уровня дохода
# и создание столбца ratio

data_pivot_total_income_group_debt['ratio'] = round(data_pivot_total_income_group_debt[1] / (
    data_pivot_total_income_group_debt[0] + data_pivot_total_income_group_debt[1]), 3)
data_pivot_total_income_group_debt.sort_values(by = 'ratio', ascending = False)

debt,0,1,ratio
total_income_group,,,
3,7709,737,0.087
2,5749,516,0.082
4,5710,453,0.074
1,545,35,0.060


**Вывод**

Согласно сводной таблице, люди, которые зарабатывают в месяц от 130000 до 200000, больше подвержены задолженности по кредиту. Следом за ними находятся люди у которых доход в месяц составляет от 55000 до 120000. Чаще всего платят по кредиту в срок люди у которых заработная плата в месяц составляет до 55000.

Следовательно, можно сказать, что зависимости между уровнем дохода и возвратом кредита в срок не существует.

- Как разные цели кредита влияют на его возврат в срок?

In [46]:
# создание сводной таблицы

data_pivot_purpose_categories_debt = df.pivot_table(index = 'purpose_categories', columns = 'debt', 
                                                    values = 'gender', aggfunc = 'count')
data_pivot_purpose_categories_debt

debt,0,1
purpose_categories,,
автомобиль,3903,403
недвижимость,10029,782
образование,3643,370
свадьба,2138,186


In [47]:
# подсчет вероятности задолженности для каждой цели получения кредита
# и создание столбца 'ratio'

data_pivot_purpose_categories_debt['ratio'] = round(data_pivot_purpose_categories_debt[1] / (
    data_pivot_purpose_categories_debt[0] + data_pivot_purpose_categories_debt[1]), 3)
data_pivot_purpose_categories_debt.sort_values(by = 'ratio', ascending = False)

debt,0,1,ratio
purpose_categories,,,
автомобиль,3903,403,0.094
образование,3643,370,0.092
свадьба,2138,186,0.080
недвижимость,10029,782,0.072


**Вывод**

Согласно сводной таблице, чаще всего платят в срок люди, которые берут кредит на недвижимость, так как заинтересованы в собственном жилье. Хуже всего платят люди, которые берут кредит на покупку автомобиля или на образование.

# 4. Общий вывод <a id='intro3'></a>

В процессе исследования для начала мы провели предобработку данных в результе которой:
* заполнили пропущенные значения;
* изменили тип данных с вещественного на целочислительный в столбце `days_employed`;
* удалили дубликаты, как явные, так и неявные;
* выделили леммы в значениях столбца `purpose`;
* категоризировали данные по возрастным группам и по доходам людей.

Далее мы ответили на `4` вопроса и выявили, что:
* Зависимость между наличием детей и возвратом кредита в срок не существует, так как люди без детей чаще выплачивают кредит в срок;
* Зависимость между семейным положением и возвратом кредита в срок существует, так как люди, которые хоть один раз состояли в официальном браке чаще выплачивают кредит в срок и не оказываются должниками перед банком;
* Зависимость между уровнем дохода и возвратом кредита в срок не существует, так как чаще всего платят в срок люди у которых заработная плата составляет до 55000 в месяц, а люди, которые зарабатывают от 130000 до 200000 в месяц наоборот более подтвержены задолженности по кредиту;
* Выплата в срок кредита на недвижимость более частое явление, люди заинтересованы в собственном жилье. Наоборот, совсем другая ситуация обстоит с кредитами на покупку автомобиля или на образование, там задолженность явление не редкое.